In [0]:
%pip install -U openai==0.28

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.9.0
    Not uninstalling openai at /local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-1cb5a838-9f4a-4f90-beb6-c4e1bbc87b60
    Can't uninstall 'openai'. No files were found to uninstall.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
%pip install tenacity

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
import os
import pandas as pd, numpy as np
import ast
import tenacity

In [0]:
import os
MY_OPENAPI_KEY = '4db7f4f08cb141669f15d67fd73bfb38'                 
os.environ["OPENAI_API_KEY"] = MY_OPENAPI_KEY
# os.environ["OPENAI_API_KEY"] = my_key

### Color  Extraction

In [0]:
### Ingest Item Data
items = pd.read_parquet('/dbfs/FileStore/tables/MALLIK/MASTER_DATA.parquet')
good_GTINs = items[(items.INACTIVE_FLAG != 'Yes')\
& (items.VERIFIED_FLAG == True)\
& (items['Sales Variant'] == 'Yes')].GTIN_NO.unique()
items = items[items.GTIN_NO.isin(good_GTINs)]

### To add sales channel flag
sls_chn_map = dict(zip(items.GTIN_NO, items.SLS_CHA_VLU))


### Incorporate CFIC fields
'''
Source Table with CF Taxonomy:   pimmart.ITM_CFIC_FYT
'''
cust_facing_df = pd.read_csv('/dbfs/FileStore/tables/DATA_SCIENCE/CUSTOMER_FACING_ITEM_DATA_112823.zip', low_memory = False)
cfic_cols = ['GTIN_NO', 'ITM_CFIC_FYT_DPT_CD', 'ITM_CFIC_FYT_COM_CD',
       'ITM_CFIC_FYT_SUB_COM_CD', 'ITM_CFIC_FYT_DPT_NAM',
       'ITM_CFIC_FYT_COM_NAM', 'ITM_CFIC_FYT_SUB_COM_NAM',
       'ITM_CFIC_FYT_CGY_CD', 'ITM_CFIC_FYT_CGY_VLU']
items = items.merge(cust_facing_df[cfic_cols], how = 'left', on = 'GTIN_NO')  

In [0]:
items.dtypes

ITM_ID                               int64
GTIN_NO                              int64
VND_BRN_NAM                         object
NET_CONT_EQV_QY_VIP_TO_KRG_QY      float64
NET_CONT_EQV_UOM_VIP_TO_KRG_UOM     object
CON_UNT_ITM_DSC                     object
PDT_FRE_FRO_CD                      object
PDT_FRE_FRO_VLU                     object
NET_CONT_VND_QY                    float64
NET_CONT_UOM_VND_CD                 object
NET_CONT_UOM_VND_VLU                object
GTIN_NAM                            object
NO_OF_SERV_PER_PKG_NO              float64
PMY_DPT_CD                         float64
PMY_DPT_DSC                         object
REC_DPT_CD                         float64
REC_DPT_DSC                         object
DPT_CD                             float64
DPT_DSC                             object
COM_CD                             float64
COM_DSC                             object
SUBCOM_CD                          float64
SUBCOM_DSC                          object
MKT_BRN_CD 

In [0]:
# List Unique CFIC COM names
sorted(items.ITM_CFIC_FYT_COM_NAM.dropna().unique().tolist())

[' Sparkling Water & Seltzer Water',
 'Action Figures & Playsets',
 'Activewear',
 'Air Fresheners',
 'Alcohol Pairings',
 'Allergy & Sinus',
 'Americana',
 'Angel Food, Pound, & Pudding Cakes',
 'Aquarium Needs',
 'Artificial Flowers, Plants, & Wreaths',
 'Artisan Breads',
 'As Seen on TV',
 'Audio & MP3 Players',
 'Automotive',
 'Baby',
 'Baby Apparel',
 'Baby Bath & Skin Care',
 'Baby Bedding & Accessories',
 'Baby Formula',
 'Baby Gates',
 'Baby Gear, Activity & Toys',
 'Baby Monitors',
 'Baby Proofing',
 'Baby Shoes',
 'Bacon & Breakfast Sausage',
 'Bakery & Bread',
 'Bakeware',
 'Baking & Cooking',
 'Baking Paper & Foils',
 'Balloons',
 'Bar & Wine Accessories',
 'Bath & Body',
 'Bath Towels and Rugs',
 'Bath Towels, Washcloths',
 'Bath Toys',
 'Bath Tubs & Accessories',
 'Bathroom Accessories',
 'Bathroom Furniture',
 'Bathroom Hardware',
 'Bathroom Mirrors And Lighting',
 'Bathroom Storage',
 'Batteries',
 'Beads & Jewelry',
 'Beauty & Personal Care',
 'Beauty Gifting',
 'Beddi

In [0]:
#SUBSET TO COMMODITY OF INTEREST
dpt_name = 'Floral'
items_dpt = items[items.ITM_CFIC_FYT_DPT_NAM == dpt_name]

# Provide one-shot examples
oneshot_dsc = 'Almay 503 Black Brown Multi-Benefit Mascara'
oneshot_output = "{'Color': 'Black;Brown', 'Confidence': 100}"

In [0]:
#Note: The openai-python library support for Azure OpenAI is in preview.
      #Note: This code sample requires OpenAI Python library version 0.28.1 or lower.
import os
import openai
import time

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

#@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def extract_attribute(input, attribute = 'color',cat = dpt_name, time_delay = 0.08):
  openai.api_type = "azure"
  openai.api_base = "https://eastus2.api.cognitive.microsoft.com/"
  openai.api_version = "2023-07-01-preview"
  openai.api_key = os.getenv("OPENAI_API_KEY")
  #openai.api_key_path = <API-KEY>
  
  message_text = [{"role":"system","content":"What is the  " + attribute + " for this " + cat +  " item?: " + input + ". Separate multiple " + attribute + " values with a / and provide the answer and the confidence percentage in a JSON-like format. If no value is detected, return 'OTHER' with a confidence of 100. If the description contains a very rare color, report the more common variant of that color. Example: If the input is 'Delphinium', return {'Color': 'BLUE', 'Confidence': 90}"}]
  time.sleep(time_delay)
  completion = openai.ChatCompletion.create(
    engine="gpt35modelfn",
    messages = message_text,
    temperature=0.05,
    max_tokens= 100,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
  )
  try:
    print(f'completion:  {completion}')
    return completion['choices'][0]['message']['content']
  except:
    return "{"+attribute+":'OTHER',Confidence:100}"
  

In [0]:
### Test output
sample_input = 'marigold'
sample_output = extract_attribute(sample_input)
print('Input:',sample_input,'\n','Response:' ,sample_output)

completion:  {
  "id": "chatcmpl-8jwrkbyTCU14dI0NNV1pfH3Ci3KXi",
  "object": "chat.completion",
  "created": 1705961732,
  "model": "gpt-35-turbo",
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{'Color': 'YELLOW/ORANGE', 'Confidence': 90}"
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "seve

In [0]:

sample_input = 'bleach'
sample_output = extract_attribute(sample_input)
print('Input:',sample_input,'\n','Response:' ,sample_output)

completion:  {
  "id": "chatcmpl-8jx8itslqufPt5B5tDqXys76t9YLh",
  "object": "chat.completion",
  "created": 1705962784,
  "model": "gpt-35-turbo",
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{'Color': 'OTHER', 'Confidence': 100}"
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": 

In [0]:
items_dpt.shape

(8091, 60)

In [0]:
### Write outputs to file
batch_size = 1000
n_chunks = int(np.ceil(len(items_dpt)/batch_size))
batch = range(1, n_chunks + 1)
for batch_id in batch:
    item_samp = items_dpt[['GTIN_NO','VND_ECOM_DSC','ITM_LVL_DSC']][1000*(batch_id -1):1000*batch_id]
    item_samp['OpenAI_Output'] = item_samp.VND_ECOM_DSC.fillna('None').apply(lambda x: extract_attribute(x))
    item_samp['OpenAI_Output'] = item_samp.OpenAI_Output.apply(lambda x: ast.literal_eval(x))
    item_samp['Color'] = item_samp.OpenAI_Output.apply(lambda x: x['Color'].upper() if x != None else 'OTHER')
    item_samp['Confidence'] = item_samp.OpenAI_Output.apply(lambda x: x['Confidence'] if x != None else 0)
    item_samp[['GTIN_NO','VND_ECOM_DSC','Color','Confidence']].to_csv('/dbfs/FileStore/tables/DATA_SCIENCE/OpenAI/Shiladitya/Color_' + dpt_name.replace(' ','_') +'SingleValue_'+ str(batch_id)+'.csv', index = None)

In [0]:
#Test
batch_id = 9
item_samp = items_dpt[['GTIN_NO','VND_ECOM_DSC','ITM_LVL_DSC']][1000*(batch_id -1):1000*batch_id]
item_samp['OpenAI_Output'] = item_samp.VND_ECOM_DSC.fillna('None').apply(lambda x: extract_attribute(x))
item_samp['OpenAI_Output'] = item_samp.OpenAI_Output.apply(lambda x: ast.literal_eval(x))
item_samp['Color'] = item_samp.OpenAI_Output.apply(lambda x: x['Color'].upper() if x != None else 'OTHER')
item_samp['Confidence'] = item_samp.OpenAI_Output.apply(lambda x: x['Confidence'] if x != None else 0)
item_samp[['GTIN_NO','VND_ECOM_DSC','Color','Confidence']].to_csv('/dbfs/FileStore/tables/DATA_SCIENCE/OpenAI/Shiladitya/Color_' + dpt_name.replace(' ','_') +'SingleValue_'+ str(batch_id)+'.csv', index = None)

In [0]:
out1 = pd.read_csv('/dbfs/FileStore/tables/DATA_SCIENCE/OpenAI/Shiladitya/Color_FloralSingleValue_1.csv')
out2 = pd.read_csv('/dbfs/FileStore/tables/DATA_SCIENCE/OpenAI/Shiladitya/Color_FloralSingleValue_2.csv')
out3 = pd.read_csv('/dbfs/FileStore/tables/DATA_SCIENCE/OpenAI/Shiladitya/Color_FloralSingleValue_3.csv')
out4 = pd.read_csv('/dbfs/FileStore/tables/DATA_SCIENCE/OpenAI/Shiladitya/Color_FloralSingleValue_4.csv')
out5 = pd.read_csv('/dbfs/FileStore/tables/DATA_SCIENCE/OpenAI/Shiladitya/Color_FloralSingleValue_5.csv')
out6 = pd.read_csv('/dbfs/FileStore/tables/DATA_SCIENCE/OpenAI/Shiladitya/Color_FloralSingleValue_6.csv')
out7 = pd.read_csv('/dbfs/FileStore/tables/DATA_SCIENCE/OpenAI/Shiladitya/Color_FloralSingleValue_7.csv')
out8 = pd.read_csv('/dbfs/FileStore/tables/DATA_SCIENCE/OpenAI/Shiladitya/Color_FloralSingleValue_8.csv')
out9 = pd.read_csv('/dbfs/FileStore/tables/DATA_SCIENCE/OpenAI/Shiladitya/Color_FloralSingleValue_9.csv')

In [0]:
pd.concat([out1, out2, out3, out4,out5,out6,out7,out8,out9]).to_csv('Color_Floral_111724.csv',index = None)